In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241204_01_RDD_API")
spark = SparkContext(conf=conf).getOrCreate()
spark

24/12/04 11:31:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/04 11:31:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local appName=241204_01_RDD_API>

In [2]:
foods = spark.parallelize([ "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면" ] )
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [3]:
#모두 메모리에 올립니다.
foods.collect() #take() show()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [4]:
#값을 기준으로 카운트 : 음식별 개수 세기
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [5]:
#상위 3개만 가져와
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [6]:
#처음 1개 가져오기
foods.first()

'짜장면'

In [7]:
#RDD 갯수
foods.count()

12

In [8]:
#중복 제거한 목록
#Transformation 연산
fd_dist = foods.distinct()
fd_dist

PythonRDD[9] at RDD at PythonRDD.scala:53

In [9]:
#action 연산 : Result
fd_dist.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [14]:
#워커노드에서 실행하는 기능
foods.foreach(lambda x : print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


In [15]:
foods.foreach(lambda x : x)

# Narrow Operation  1:1 연산 
filter(), map(), flatMap(), sample(), union()

In [17]:
sample_rdd = spark.parallelize([1,2,3,4,5])

In [18]:
sample_rdd2 = sample_rdd.map(lambda x : x+2) #d+2, 2+2, 3+2....

In [19]:
sample_rdd2.collect()

[3, 4, 5, 6, 7]

In [21]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [22]:
moviesRDD = spark.parallelize(movies)

In [23]:
mapMovies = moviesRDD.map(lambda x: x.split(" "))

In [24]:
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [25]:
flatMapMovies = moviesRDD.flatMap(lambda x:x.split(" ") )
flatMapMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [29]:
filteredMovies = flatMapMovies.filter(lambda x: x!="매트릭스")
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [30]:
#집합연산
num1 = spark.parallelize([1,2,3,4,5])
num2 = spark.parallelize([4,5,6,7,8,9,10])

In [31]:
#intersection
num1.intersection(num2).collect()

[4, 5]

In [ ]:
#union, subtract

# sample(withReplacement, fraction, seed)

일부를 샘플링해서 추출한다.
```
withReplacement : 비복원, 복원 추출
fraction : 기대값
seed : 난수추출을 위한 시드값 - 재현 가능
```

In [32]:
numlist = num1.union(num2)
numlist.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [33]:
numlist.sample(True, 0.5).collect()

[1, 1, 5, 5, 4, 6, 8]

In [43]:
numlist.sample(True, 0.3).collect()

[4, 4, 5, 6, 6]

In [40]:
numlist.sample(False, 0.7).collect() #재현이 안되는 샘플

[2, 3, 4, 5, 6, 8, 9, 10]

In [42]:
numlist.sample(True, 0.5, seed=42).collect() #재현 가능한 샘플

[5, 6, 6]

# wide transformation
groupby reduce.. 

In [44]:
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [46]:
foodsGroup = foods.groupBy(lambda x: x[0])
res = foodsGroup.collect()

In [48]:
for (k,v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [49]:
spark.stop()

In [26]:
movies = [
"The Shawshank Redemption", "The Godfather", "The Dark Knight", "12 Angry Men",
"Schindler's List", "The Lord of the Rings: The Return of the King", "Pulp Fiction",
"The Good, the Bad and the Ugly", "Fight Club", "Forrest Gump", "Inception",
"Star Wars: Episode V - The Empire Strikes Back", "The Matrix", "Goodfellas",
"One Flew Over the Cuckoo's Nest", "Se7en", "It's a Wonderful Life", "Seven Samurai",
"The Silence of the Lambs", "City of God", "Saving Private Ryan", "Interstellar",
"The Green Mile", "Spirited Away", "Parasite", "Léon: The Professional",
"The Usual Suspects", "Harakiri", "The Lion King", "Back to the Future",
"The Pianist", "Terminator 2: Judgment Day", "American History X", "Modern Times",
"Psycho", "Gladiator", "City Lights", "The Departed", "The Intouchables",
"Whiplash", "The Prestige", "Casablanca", "Once Upon a Time in the West",
"Rear Window", "Cinema Paradiso", "Alien", "Apocalypse Now", "Memento",
"Indiana Jones and the Raiders of the Lost Ark", "The Great Dictator",
"The Lives of Others", "Django Unchained", "Paths of Glory", "The Shining",
"WALL·E", "Sunset Blvd.", "Witness for the Prosecution", "Oldboy", "Princess Mononoke",
"Dr. Strangelove", "The Dark Knight Rises", "Once Upon a Time in America",
"Aliens", "Coco", "Avengers: Infinity War", "American Beauty", "Braveheart",
"Das Boot", "3 Idiots", "Toy Story", "Amadeus", "Inglourious Basterds",
"Good Will Hunting", "Star Wars: Episode IV - A New Hope", "Reservoir Dogs",
"2001: A Space Odyssey", "Vertigo", "M", "Citizen Kane", "Requiem for a Dream",
"The Hunt", "Eternal Sunshine of the Spotless Mind", "Full Metal Jacket",
"A Clockwork Orange", "Taxi Driver", "Lawrence of Arabia", "Double Indemnity",
"Amélie", "Like Stars on Earth", "Singin' in the Rain", "Bicycle Thieves",
"Snatch", "Scarface", "3 Idiots", "Toy Story 3", "The Sting", "Metropolis",
"For a Few Dollars More", "L.A. Confidential", "Rashomon", "Monty Python and the Holy Grail",
"The Shawshank Redemption", "The Godfather", "The Dark Knight", "12 Angry Men",
"Schindler's List", "The Lord of the Rings: The Return of the King", "Pulp Fiction",
"The Good, the Bad and the Ugly", "Fight Club", "Forrest Gump", "Inception",
"Star Wars: Episode V - The Empire Strikes Back", "The Matrix", "Goodfellas",
"One Flew Over the Cuckoo's Nest", "Se7en", "It's a Wonderful Life", "Seven Samurai",
"The Silence of the Lambs", "City of God", "Saving Private Ryan", "Interstellar",
"The Green Mile", "Spirited Away", "Parasite", "Léon: The Professional",
"The Usual Suspects", "Harakiri", "The Lion King", "Back to the Future",
"The Pianist", "Terminator 2: Judgment Day", "American History X", "Modern Times",
"Psycho", "Gladiator", "City Lights", "The Departed", "The Intouchables",
"Whiplash", "The Prestige", "Casablanca", "Once Upon a Time in the West",
"Rear Window", "Cinema Paradiso", "Alien", "Apocalypse Now", "Memento",
"Indiana Jones and the Raiders of the Lost Ark", "The Great Dictator",
"The Lives of Others", "Django Unchained", "Paths of Glory", "The Shining",
"WALL·E", "Sunset Blvd.", "Witness for the Prosecution", "Oldboy", "Princess Mononoke",
"Dr. Strangelove", "The Dark Knight Rises", "Once Upon a Time in America",
"Aliens", "Coco", "Avengers: Infinity War", "American Beauty", "Braveheart",
"Das Boot", "3 Idiots", "Toy Story", "Amadeus", "Inglourious Basterds",
"Good Will Hunting", "Star Wars: Episode IV - A New Hope", "Reservoir Dogs",
"2001: A Space Odyssey", "Vertigo", "M", "Citizen Kane", "Requiem for a Dream",
"The Hunt", "Eternal Sunshine of the Spotless Mind", "Full Metal Jacket",
"A Clockwork Orange", "Taxi Driver", "Lawrence of Arabia", "Double Indemnity",
"Amélie", "Like Stars on Earth", "Singin' in the Rain", "Bicycle Thieves",
"Snatch", "Scarface", "3 Idiots", "Toy Story 3", "The Sting", "Metropolis",
"For a Few Dollars More", "L.A. Confidential", "Rashomon", "Monty Python and the Holy Grail","The Shawshank Redemption", "The Godfather", "The Dark Knight", "12 Angry Men",
"Schindler's List", "The Lord of the Rings: The Return of the King", "Pulp Fiction",
"The Good, the Bad and the Ugly", "Fight Club", "Forrest Gump", "Inception",
"Star Wars: Episode V - The Empire Strikes Back", "The Matrix", "Goodfellas",
"One Flew Over the Cuckoo's Nest", "Se7en", "It's a Wonderful Life", "Seven Samurai",
"The Silence of the Lambs", "City of God", "Saving Private Ryan", "Interstellar",
"The Green Mile", "Spirited Away", "Parasite", "Léon: The Professional",
"The Usual Suspects", "Harakiri", "The Lion King", "Back to the Future",
"The Pianist", "Terminator 2: Judgment Day", "American History X", "Modern Times",
"Psycho", "Gladiator", "City Lights", "The Departed", "The Intouchables",
"Whiplash", "The Prestige", "Casablanca", "Once Upon a Time in the West",
"Rear Window", "Cinema Paradiso", "Alien", "Apocalypse Now", "Memento",
"Indiana Jones and the Raiders of the Lost Ark", "The Great Dictator",
"The Lives of Others", "Django Unchained", "Paths of Glory", "The Shining",
"WALL·E", "Sunset Blvd.", "Witness for the Prosecution", "Oldboy", "Princess Mononoke",
"Dr. Strangelove", "The Dark Knight Rises", "Once Upon a Time in America",
"Aliens", "Coco", "Avengers: Infinity War", "American Beauty", "Braveheart",
"Das Boot", "3 Idiots", "Toy Story", "Amadeus", "Inglourious Basterds",
"Good Will Hunting", "Star Wars: Episode IV - A New Hope", "Reservoir Dogs",
"2001: A Space Odyssey", "Vertigo", "M", "Citizen Kane", "Requiem for a Dream",
"The Hunt", "Eternal Sunshine of the Spotless Mind", "Full Metal Jacket",
"A Clockwork Orange", "Taxi Driver", "Lawrence of Arabia", "Double Indemnity",
"Amélie", "Like Stars on Earth", "Singin' in the Rain", "Bicycle Thieves",
"Snatch", "Scarface", "3 Idiots", "Toy Story 3", "The Sting", "Metropolis",
"For a Few Dollars More", "L.A. Confidential", "Rashomon", "Monty Python and the Holy Grail","The Shawshank Redemption", "The Godfather", "The Dark Knight", "12 Angry Men",
"Schindler's List", "The Lord of the Rings: The Return of the King", "Pulp Fiction",
"The Good, the Bad and the Ugly", "Fight Club", "Forrest Gump", "Inception",
"Star Wars: Episode V - The Empire Strikes Back", "The Matrix", "Goodfellas",
"One Flew Over the Cuckoo's Nest", "Se7en", "It's a Wonderful Life", "Seven Samurai",
"The Silence of the Lambs", "City of God", "Saving Private Ryan", "Interstellar",
"The Green Mile", "Spirited Away", "Parasite", "Léon: The Professional",
"The Usual Suspects", "Harakiri", "The Lion King", "Back to the Future",
"The Pianist", "Terminator 2: Judgment Day", "American History X", "Modern Times",
"Psycho", "Gladiator", "City Lights", "The Departed", "The Intouchables",
"Whiplash", "The Prestige", "Casablanca", "Once Upon a Time in the West",
"Rear Window", "Cinema Paradiso", "Alien", "Apocalypse Now", "Memento",
"Indiana Jones and the Raiders of the Lost Ark", "The Great Dictator",
"The Lives of Others", "Django Unchained", "Paths of Glory", "The Shining",
"WALL·E", "Sunset Blvd.", "Witness for the Prosecution", "Oldboy", "Princess Mononoke",
"Dr. Strangelove", "The Dark Knight Rises", "Once Upon a Time in America",
"Aliens", "Coco", "Avengers: Infinity War", "American Beauty", "Braveheart",
"Das Boot", "3 Idiots", "Toy Story", "Amadeus", "Inglourious Basterds",
"Good Will Hunting", "Star Wars: Episode IV - A New Hope", "Reservoir Dogs",
"2001: A Space Odyssey", "Vertigo", "M", "Citizen Kane", "Requiem for a Dream",
"The Hunt", "Eternal Sunshine of the Spotless Mind", "Full Metal Jacket",
"A Clockwork Orange", "Taxi Driver", "Lawrence of Arabia", "Double Indemnity",
"Amélie", "Like Stars on Earth", "Singin' in the Rain", "Bicycle Thieves",
"Snatch", "Scarface", "3 Idiots", "Toy Story 3", "The Sting", "Metropolis",
"For a Few Dollars More", "L.A. Confidential", "Rashomon", "Monty Python and the Holy Grail"    
]

In [27]:
moviesRDD = spark.parallelize(movies)
moviesRDD

ParallelCollectionRDD[18] at readRDDFromFile at PythonRDD.scala:274

In [28]:
moviesRDD.collect()

['The Shawshank Redemption',
 'The Godfather',
 'The Dark Knight',
 '12 Angry Men',
 "Schindler's List",
 'The Lord of the Rings: The Return of the King',
 'Pulp Fiction',
 'The Good, the Bad and the Ugly',
 'Fight Club',
 'Forrest Gump',
 'Inception',
 'Star Wars: Episode V - The Empire Strikes Back',
 'The Matrix',
 'Goodfellas',
 "One Flew Over the Cuckoo's Nest",
 'Se7en',
 "It's a Wonderful Life",
 'Seven Samurai',
 'The Silence of the Lambs',
 'City of God',
 'Saving Private Ryan',
 'Interstellar',
 'The Green Mile',
 'Spirited Away',
 'Parasite',
 'Léon: The Professional',
 'The Usual Suspects',
 'Harakiri',
 'The Lion King',
 'Back to the Future',
 'The Pianist',
 'Terminator 2: Judgment Day',
 'American History X',
 'Modern Times',
 'Psycho',
 'Gladiator',
 'City Lights',
 'The Departed',
 'The Intouchables',
 'Whiplash',
 'The Prestige',
 'Casablanca',
 'Once Upon a Time in the West',
 'Rear Window',
 'Cinema Paradiso',
 'Alien',
 'Apocalypse Now',
 'Memento',
 'Indiana Jones

In [50]:
spark.stop()